In [126]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


# Подготовка контекста

In [181]:
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

with open("/home/jovyan/work/docs/story.md") as f:
    text_story = f.read()
    
chunks_story = text_splitter.split_text(text_story)
documents_story = [Document(page_content=text) for text in chunks_story]
# len(chunks_story)
len(documents_story)
# documents_story[:3]

95

## Векторизуем основной контекст Ollama

In [182]:
from ollama import Client
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from typing import List

client = Client(host='http://host.docker.internal:11434')

texts = [doc.page_content for doc in documents_story]

# Получаем эмбеддинги
class OllamaEmbeddings(Embeddings):
    def __init__(self, model: str, host: str):
        self.client = Client(host=host)
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        results = []
        for txt in texts:
            start = time.time()
            embedding = self.client.embeddings(model=self.model, prompt=txt)["embedding"]
            print(f"Embedded in {time.time() - start:.2f}s: {txt[:30]}...")
            results.append(embedding)
        return results

    def embed_query(self, text: str) -> List[float]:
        return self.client.embeddings(model=self.model, prompt=text)["embedding"]

# Пример:
ollama_embedder = OllamaEmbeddings(model="mxbai-embed-large", host="http://host.docker.internal:11434")
vectorstore = FAISS.from_documents(documents_story, ollama_embedder)

Embedded in 0.31s: Путешествия Зелёного Человечка...
Embedded in 0.03s: В 1980 году с планеты Зурбле-7...
Embedded in 0.03s: или просто Зелёч, в экспедицию...
Embedded in 0.04s: изучить обитателей изолированн...
Embedded in 0.02s: культуру, язык и абсурдную лог...
Embedded in 0.03s: # Остров 1: Чайнофия

1980

“К...
Embedded in 0.04s: Зелёч прибыл на остров, утопаю...
Embedded in 0.03s: Воздух здесь был горячим, но н...
Embedded in 0.03s: Завариссы — высокие, тонкие су...
Embedded in 0.03s: вместо ушей. Говорят они на вз...
Embedded in 0.02s: Они не спят, а погружаются в т...
Embedded in 0.03s: Общество у них матриархальное:...
Embedded in 0.02s: поэмы для листьев....
Embedded in 0.03s: Зелёч выучил их церемонию паро...
Embedded in 0.03s: # Остров 2: Бюрократика

1982
...
Embedded in 0.03s: Пейзаж острова — бесконечные о...
Embedded in 0.02s: названы по типам заявлений....
Embedded in 0.03s: Формуляры — приземистые сущест...
Embedded in 0.03s: Из ушей свисают синие ручки. Р...
Embedded

In [189]:
query = "что было в 1995 году?"
query_vector = ollama_embedder.embed_query(query)
retrieved_docs = vectorstore.similarity_search_with_score_by_vector(query_vector)

for doc, score in retrieved_docs:
    print(f"Score: {score:.4f}\nContent: {doc.page_content[:200]}\n{'-'*50}")
    
context_text = "\n\n".join([doc.page_content for doc, score in retrieved_docs])

Score: 158.0827
Content: # Остров 6: Лентиленд

1995

“Где каждый день — понедельник”
--------------------------------------------------
Score: 197.9559
Content: В 1980 году с планеты Зурбле-7 был отправлен исследователь — Зелёный Человечек, или просто Зелёч, в
--------------------------------------------------
Score: 219.2520
Content: Воздух здесь был горячим, но насыщал, словно бульон.
--------------------------------------------------
Score: 225.6282
Content: Всё здесь происходило медленно. Даже свет был слегка замедленным и тёплым, словно полдень в 2007
--------------------------------------------------


## Интерпритация через LLM

In [190]:
# Ты — искусственный интеллект, который читает книгу Льва Толстого Война и Мир
prompt = f"""
Ты — искусственный интеллект, который читает исторические документы о планете Сберля. 

Вот вопрос пользователя:
"{query}"

Вот выдержки из документов, релевантные вопросу:
{context_text}

На основе приведённых документов:
1. Ответь расширенно на вопрос пользователя.
2. Если ответа нет, однозначно скажи, что в предоставленных текстах об этом не сказано.
3. Не выдумывай фактов вне этих документов.

Ответ:
"""

response = client.chat(
    model="llama3.2",
    messages=[
        {"role": "user", "content": prompt}
    ]
)
print(response['message']['content'])

Из предоставленных документов о планете Сберля в 1995 году नहодит nothing особых событий или информации, связанной напрямую с этой дата. Однако, существует упоминание 1980 года, когда на Зурбле-7 был отправлен исследователь - Зелёный Человек (Зелёч), который прибыл на Лентиленд в 1995 году.


# Обогащение контекста